In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from IPython.display import display

# Customizable variables (adjust these as needed)
INITIAL_INVESTMENT = 10000  # Fixed initial investment in USD
FIXED_PERCENTAGE = 0.02     # Fixed % of portfolio to invest monthly for DCA (e.g., 2%)
MONTHLY_INVESTMENT = 100    # Fixed dollar amount for DCA each month
START_DATE = '1971-02-05'   # Earliest launch date (Nasdaq start; SPX is earlier)
END_DATE = '2025-02-25'     # Current date
RISK_FREE_RATE = 0.02       # Annual risk-free rate for Sharpe ratio

# Load CRSP monthly stock file (assuming it contains S&P 500 and Nasdaq data)
msf_data = pd.read_parquet('crsp.msf.parquet')

# Filter for S&P 500 (^GSPC) and Nasdaq (^IXIC) index data
# Note: CRSP may use index identifiers (e.g., 10001 for S&P 500); adjust if needed
spx_data = msf_data[msf_data['TICKER'] == '^GSPC'].copy()  # S&P 500
ixic_data = msf_data[msf_data['TICKER'] == '^IXIC'].copy()  # Nasdaq

# Ensure date is datetime and sort
spx_data['DATE'] = pd.to_datetime(spx_data['DATE'])
ixic_data['DATE'] = pd.to_datetime(ixic_data['DATE'])
spx_data = spx_data.sort_values('DATE')
ixic_data = ixic_data.sort_values('DATE')

# Filter data by date range
spx_data = spx_data[(spx_data['DATE'] >= START_DATE) & (spx_data['DATE'] <= END_DATE)]
ixic_data = ixic_data[(ixic_data['DATE'] >= START_DATE) & (ixic_data['DATE'] <= END_DATE)]

# Use adjusted price (PRC) or returns (RET); assuming PRC is present and positive
spx_prices = spx_data[['DATE', 'PRC']].set_index('DATE')
ixic_prices = ixic_data[['DATE', 'PRC']].set_index('DATE')

# Handle missing or negative prices (CRSP uses negative PRC for bid/ask average)
spx_prices['PRC'] = spx_prices['PRC'].abs()
ixic_prices['PRC'] = ixic_prices['PRC'].abs()

# Function to calculate portfolio metrics
def calculate_metrics(prices, portfolio_value):
    returns = portfolio_value.pct_change().dropna()
    sharpe_ratio = (returns.mean() * 12 - RISK_FREE_RATE) / (returns.std() * np.sqrt(12))
    cumulative_return = (portfolio_value[-1] / portfolio_value[0]) - 1
    rolling_max = portfolio_value.cummax()
    drawdown = (portfolio_value - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
    return sharpe_ratio, cumulative_return, max_drawdown, drawdown

# 1. Buy and Hold Strategy
def buy_and_hold(prices, initial_investment):
    shares = initial_investment / prices['PRC'].iloc[0]
    portfolio_value = shares * prices['PRC']
    return portfolio_value

# 2. Dollar-Cost Averaging Strategy
def dollar_cost_averaging(prices, monthly_investment):
    shares = 0
    portfolio_value = pd.Series(index=prices.index, dtype=float)
    for date, price in prices['PRC'].items():
        shares += monthly_investment / price
        portfolio_value[date] = shares * price
    return portfolio_value

# Simulate portfolios
strategies = {
    'SPX Buy and Hold': buy_and_hold(spx_prices, INITIAL_INVESTMENT),
    'SPX DCA': dollar_cost_averaging(spx_prices, MONTHLY_INVESTMENT),
    'IXIC Buy and Hold': buy_and_hold(ixic_prices, INITIAL_INVESTMENT),
    'IXIC DCA': dollar_cost_averaging(ixic_prices, MONTHLY_INVESTMENT)
}

# Calculate metrics and store results
results = {}
for name, portfolio in strategies.items():
    sharpe, cum_return, max_dd, drawdown = calculate_metrics(spx_prices if 'SPX' in name else ixic_prices, portfolio)
    results[name] = {'Sharpe Ratio': sharpe, 'Cumulative Return': cum_return, 'Max Drawdown': max_dd}
    strategies[name] = portfolio  # Update with full series for plotting

# Display results in a table
results_df = pd.DataFrame(results).T
display(results_df)

# Visualization
plt.figure(figsize=(15, 10))

# 1. Cumulative Portfolio Value
plt.subplot(3, 1, 1)
for name, portfolio in strategies.items():
    plt.plot(portfolio, label=name)
plt.title('Cumulative Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid()

# 2. Cumulative Returns
plt.subplot(3, 1, 2)
for name, portfolio in strategies.items():
    cum_returns = (portfolio / portfolio.iloc[0]) - 1
    plt.plot(cum_returns, label=name)
plt.title('Cumulative Returns Over Time')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid()

# 3. Drawdown
plt.subplot(3, 1, 3)
for name, portfolio in strategies.items():
    _, _, _, drawdown = calculate_metrics(spx_prices if 'SPX' in name else ixic_prices, portfolio)
    plt.plot(drawdown, label=name)
plt.title('Drawdown Over Time')
plt.xlabel('Date')
plt.ylabel('Drawdown')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()